# 可以走通的示例程序

In [ ]:
# coding=utf-8
# from pyspark.sql import HiveContext Row

from pyspark import SparkConf, SparkContext, HiveContext

from pyspark.sql import SparkSession, HiveContext

def spark_hive2():
    master="local"  #yarn
    conf=SparkConf().setAppName("ManageReport").setMaster(master)
    sc=SparkContext(conf=conf)
    # sc.setCheckpointDir(u"")
    hive_context=HiveContext(sc)

    path = "hdfs://172.172.172.166:8020/tmp/test/spark-test.csv"

    financeDF = hive_context.read.csv(path, header=True,)
    # financeDF.printSchema()

    financeDF.registerTempTable("finance")
    financeBasicDF = hive_context.sql("""select * from finance limit 10""")
    financeBasicDF.show()

    financeBasicDF.write.mode("overwrite").saveAsTable("finance_hive")
    financeBasicDF.show()
    hive_context.sql(""" select * from finance_hive""").show()



def spark_hive1():
    master="local"  #yarn
    conf=SparkConf().setAppName("ManageReport").setMaster(master)
    sc=SparkContext(conf=conf)
    # sc.setCheckpointDir(u"")
    hive_context=HiveContext(sc)

    path = "hdfs://172.172.172.166:8020/tmp/test/spark-test.csv"

    financeDF = hive_context.read.csv(path, header=True)
    # financeDF.printSchema()

    financeDF.createOrReplaceTempView("finance")
    financeBasicDF = hive_context.sql("""select * from finance """)
    # financeBasicDF.show()
    # financeBasicDF.cache()
    # financeBasicDF.registerTempTable('finance_tepm')

    # spark is an existing SparkSession
    # spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING)")
    # spark.sql("LOAD DATA LOCAL INPATH 'examples/src/main/resources/kv1.txt' INTO TABLE src")
    # spark.sql("CREATE TABLE IF NOT EXISTS tablea (key INT, value STRING)")
    # spark.sql("""select * from finance_tepm into TABLE tablea""")
 # partitionBy 的分区字段的内容不能含有中文 (不识别中文)
    financeBasicDF.write.mode('overwrite').saveAsTable('finance_hive',partitionBy='id')
    financeBasicDF.show()
    # hive_context.sql(""" select * from finance_hive""").show()

def spark_hive():
    # warehouse_location points to the default location for managed databases and tables
    # warehouse_location = 'spark-warehouse'

    spark = SparkSession \
        .builder \
        .appName("Py12122'1''thon Spark SQL Hive integration example") \
        .config("spark.sql.warehouse.dir", "warehouse_location") \
        .enableHiveSupport() \
        .getOrCreate()

    # path = "/root/Documents/spark-test.csv"
    # path = "/root/Documents/spark-test.csv"
    path = "hdfs://172.172.172.166:8020/tmp/test/spark-test.csv"

    financeDF = spark.read.csv(path, header=True)
    # financeDF.printSchema()

    financeDF.createOrReplaceTempView("finance")
    financeBasicDF = spark.sql("""select * from finance limit 10""")
    financeBasicDF.show()
    # financeBasicDF.cache()
    financeBasicDF.registerTempTable('finance_tepm')

    # spark is an existing SparkSession
    # spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING)")
    # spark.sql("LOAD DATA LOCAL INPATH 'examples/src/main/resources/kv1.txt' INTO TABLE src")
    # spark.sql("CREATE TABLE IF NOT EXISTS tablea (key INT, value STRING)")
    # spark.sql("""select * from finance_tepm into TABLE tablea""")

    # partitionBy 的分区字段的内容不能含有中文 (不识别中文)
    financeBasicDF.write.mode('overwrite').saveAsTable('finance_hive1',partitionBy='date')
    # financeBasicDF.write.mode('overwrite').saveAsTable('finance_hive1', partitionBy=['id','date'])
    financeBasicDF.show()
    # spark.sql(""" select * from finance_hive""").show()



#
# def spark_init():
#
#
#     spark = SparkSession \
#         .builder \
#         .appName("Python Spark SQL LEADER REPORT") \
#         .config("spark.some.config.option", "some-value") \
#         .getOrCreate()
#
#     sc = spark.sparkContext
#     # hiveContext=HiveContext(sc)
#     # hiveContext.sql("use DataBaseName")
#
#     # hdfs上文件存放路径
#     # hdfs_file_path = "hdfs://172.172.172.166:8020/user/qingchen.xue/hbase_data/"
#     path = "/root/Documents/spark-test.csv"
#     print path
#     #path = "../spark-test.csv"
#
#     financeDF = spark.read.csv(path, header=True)
#     financeDF.printSchema()
#
#     financeDF.createOrReplaceTempView("finance")
#     # financeDF.createGlobalTempView("finance")
#
#
#     financeBasicDF = spark.sql(""" with aa as ( select date_format(add_months(regexp_replace(date,"/","-"),-3),"yyyy")  as `财年`
#         ,date_format(cast(regexp_replace(date,"/","-") as date),"yyyy-MM") as `月份`
#         ,date_format(add_months(regexp_replace(date,"/","-"),-12),"yyyy-MM") as `去年同期月份`
#         ,date `日期`,area `区域`,data_type `数据类型`,substring(data_type,5) `数据类型2`,
#         sum( case when subject_type='收入' then money else 0 end ) as `全收入`,
#         sum( case when subject_type='收入' and subject_code='60011108' then money else 0 end ) as `齿科双算`, -- 60011108收入
#         sum( case when subject_detail_type='体检收入' then money else 0 end ) as `体检收入`,
#         sum( case when subject_code='64010201' or subject_code='64010202' then money else 0 end ) as  `变动成本1` ,   --64010201 and 64010202
#         sum( case when subject_detail_type='疾病检测收入' then money else 0 end ) as `疾病检测收入`,
#         sum( case when subject_detail_type='齿科收入' then money else 0 end ) as `齿科收入`,
#         sum( case when subject_code='64010221' then money else 0 end ) as `变动成本3`   ,  --64010221L
#         sum( case when subject_detail_type='门诊收入' then money else 0 end ) as `门诊收入`,
#         sum( case when subject_code='64010222' then money else 0 end ) as `变动成本4`  , --64010222
#         sum( case when subject_detail_type='医疗管理收入' then money else 0 end ) as `医疗管理收入`,
#         sum( case when subject_code='64010211' then money else 0 end ) as `变动成本5`,   --64010211
#         sum( case when subject_detail_type='销售商品收入' then money else 0 end ) as `销售商品收入`,
#         sum( case when subject_detail_type='其他收入' then money else 0 end ) as `其他收入`,
#         sum( case when subject_detail_type='落关联成本' then money else 0 end ) as `落关联成本`,
#         sum( case when subject_type='成本费用' then money else 0 end ) as `成本费用`,
#         nvl( sum( case when subject_type='收入' then money else 0 end ),0)-nvl(sum( case when subject_type='收入' and subject_code='60011108' then money else 0 end ),0) as `收入`,
#         nvl( sum( case when subject_type='收入' then money else 0 end ),0)-nvl(sum( case when subject_type='成本费用' then money else 0 end ) ,0) as `税前利润`,
#         case when area in ('西康','华检','元化医疗','健维管理','臻景','香港','BVI') then '非一体化'  else '一体化' end as `是否一体化`
#         from finance
#         group by date_format(add_months(regexp_replace(date,"/","-"),-3),"yyyy")
#             ,date_format(cast(regexp_replace(date,"/","-") as date),"yyyy-MM")
#             ,date_format(add_months(regexp_replace(date,"/","-"),-12),"yyyy-MM")
#             ,date ,area ,data_type,substring(data_type,5)
#             ,case when area in ('西康','华检','元化医疗','健维管理','臻景','香港','BVI') then '非一体化' else '一体化' end
#         )
#         ,a1 as ( --今年和去年同期 处理为一条
#         select
#         a1.`财年`,a1.`月份`,a1.`区域`,a1.`数据类型`,a1.`数据类型2`,a1.`是否一体化`,
#         a1.`体检收入`,a1.`变动成本1`,a1.`疾病检测收入`,a1.`齿科收入`,a1.`变动成本3`,a1.`门诊收入`,a1.`变动成本4`,a1.`医疗管理收入`,
#         a1.`变动成本5`,a1.`销售商品收入`,a1.`其他收入`,a1.`落关联成本`,a1.`收入`,
#         a0.`体检收入` as `去年同期体检收入`,  a0.`变动成本1` as `去年同期变动成本1`, a0.`疾病检测收入` as `去年同期疾病检测收入`,
#         a0.`齿科收入` as `去年同期齿科收入`,  a0.`变动成本3` as `去年同期变动成本3`, a0.`门诊收入` as `去年同期门诊收入`,
#         a0.`变动成本4`  as `去年同期变动成本4`,   a0.`医疗管理收入`  as `去年同期医疗管理收入`,a0.`变动成本5` as `去年同期变动成本5`,
#         a1.`销售商品收入`as `去年同期销售商品收入`,a1.`其他收入` as `去年同期其他收入`,    a1.`落关联成本` as `去年同期落关联成本`,
#         a1.`收入` as `去年同期收入`
#         from aa a1
#         left join aa a0 on a1.`区域`=a0.`区域` and a1.`数据类型2`=a0.`数据类型2` and a1.`去年同期月份`=a0.`月份`
#         )
#         select * from a1 limit 2
#         --后续添加各种逻辑sql
#         """)
#
#     financeBasicDF.show()
# #     # financeBasicDF.createGlobalTempView("financeBasic")
#     financeBasicDF.saveAsTable('financeBasic')
#
# # partitionBy 的分区字段的内容不能含有中文 (不识别中文)
#

# def read_spark():
#     sc=spark.sparkContext
#     path="../../spark-test.csv"
#     print path

# financeDF=spark.read.csv(path)
#peopleDF.printSchema()

# financeDF.createGlobalTempView("finance")





if __name__ == '__main__':
    spark_hive()